### Install keypoint MoSeq

In [ ]:
! pip install keypoint-moseq

# Import libraries

In [ ]:
import os
import requests, zipfile
import json
import numpy as np
import pandas as pd
import keypoint_moseq as kpms

# Project setup
Create a new project directory with a keypoint-MoSeq `config.yml` file.

In [ ]:
data_dir = './data'
os.makedirs(data_dir, exist_ok=True)
project_dir = './results'
os.makedirs(project_dir, exist_ok=True)

### Option: Manual setup

In [ ]:
bodyparts=['nose', 'left ear', 'right ear', 'neck', 'left hip', 'right hip', 'tail base']

skeleton=[
     ['tail base', 'left hip'],
     ['tail base', 'right hip'],
     ['right hip', 'neck'],
     ['left hip', 'neck'],
     ['left ear', 'neck'],
     ['right ear', 'neck'],
     ['left ear', 'nose'],
     ['right ear', 'nose']]

video_dir = os.path.join(data_dir, 'videos')

kpms.setup_project(
     project_dir,
     video_dir=video_dir,
     bodyparts=bodyparts,
     skeleton=skeleton,
     overwrite=True)

## Edit the config file

The config can be edited in a text editor or using the function `kpms.update_config`, as shown below. In general, the following parameters should be specified for each project:

- `bodyparts` (name of each keypoint; automatically imported from SLEAP/DeepLabCut)
- `use_bodyparts` (subset of bodyparts to use for modeling, set to all bodyparts by default; for mice we recommend excluding the tail)
- `anterior_bodyparts` and `posterior_bodyparts` (used for rotational alignment)
- `video_dir` (directory with videos of each experiment)

Edit the config as follows for the [example DeepLabCut dataset](https://drive.google.com/drive/folders/1UNHQ_XCQEKLPPSjGspRopWBj6-YNDV6G?usp=share_link):

In [ ]:
kpms.update_config(
    project_dir,
    video_dir=os.path.join(data_dir, 'videos'),
    anterior_bodyparts=['nose'],
    posterior_bodyparts=['tail base'],
    use_bodyparts=['nose', 'left ear', 'right ear', 'neck', 'left hip', 'right hip', 'tail base'],
    overwrite=True
    )
config = lambda: kpms.load_config(project_dir)

## Download the dataset 📲

The CalMS21 dataset is hosted by [Caltech](https://data.caltech.edu/records/1991). For now, we'll focus on the Task 1 data, which can be downloaded as follows:

In [ ]:
# Download and unzip the data
fname = 'task1_classic_classification.zip'
url = "https://data.caltech.edu/records/s0vdx-0k302/files/task1_classic_classification.zip?download=1"

if not os.path.isfile(fname):
  try:
    r = requests.get(url)
  except requests.ConnectionError:
    print("!!! Failed to download data !!!")
  else:
    if r.status_code != requests.codes.ok:
      print("!!! Failed to download data !!!")
    else:
      with open(fname, "wb") as fid:
        fid.write(r.content)
else:
  print('Data have already been downloaded!!!')

if not os.path.exists('task1_classic_classification'):
  # Unzip the file
  with zipfile.ZipFile(fname, 'r') as zip_ref:
    zip_ref.extractall('.')


# Download the script
fname = 'calms21_convert_to_npy.py'
url = "https://data.caltech.edu/records/s0vdx-0k302/files/calms21_convert_to_npy.py?download=1"

if not os.path.isfile(fname):
  try:
    r = requests.get(url)
  except requests.ConnectionError:
    print("!!! Failed to download data !!!")
  else:
    if r.status_code != requests.codes.ok:
      print("!!! Failed to download data !!!")
    else:
      with open(fname, "wb") as fid:
        fid.write(r.content)

The dataset files are stored as json files. For ease of handling, we'll first convert them to .npy files using the script we just downloaded, `calms21_convert_to_npy.py`. The output of this script is a pair of files, `calms21_task1_train.npy` and `calms21_task1_test.npy`.

If you include the optional `parse_treba` flag, the script will create files `calms21_task1_train_features.npy` and `calms21_task1_test_features.npy`, which contain 32 features created using <a href=https://openaccess.thecvf.com/content/CVPR2021/html/Sun_Task_Programming_Learning_Data_Efficient_Behavior_Representations_CVPR_2021_paper.html>Task Programming</a>.



In [ ]:
!python calms21_convert_to_npy.py  --input_directory '.' --output_directory '.'

## Load the data

The following loader function can be used to unpack the `.npy` files containing your train and test sets.

In [ ]:
def load_task1_data(data_path):
  """
  Load data for task 1:
      The vocaubulary tells you how to map behavior names to class ids;
      it is the same for all sequences in this dataset.
  """
  data_dict = np.load(data_path, allow_pickle=True).item()
  dataset = data_dict['annotator-id_0']
  # Get any sequence key.
  sequence_id = list(data_dict['annotator-id_0'].keys())[0]
  vocabulary = data_dict['annotator-id_0'][sequence_id]['metadata']['vocab']
  return dataset, vocabulary

In [ ]:
# Load full dataset
training_data_full, vocab = load_task1_data('./calms21_task1_train.npy')
test_data_full, _ = load_task1_data('./calms21_task1_test.npy')

# Select 
n_train = 50
n_test = 10

training_keys = list(training_data_full.keys())
training_data = {k: training_data_full[k] for k in training_keys[:n_train]}

test_keys = list(test_data_full.keys())
test_data = {k: test_data_full[k] for k in test_keys[:n_test]}


### Dataset Specifications

`training_data` and `test_data` are both dictionaries with a key for each Sequence in the dataset, where a Sequence is a single resident-intruder assay. Each Sequence contains the following fields:

<ul>
<li><b>keypoints</b>: tracked locations of body parts on the two interacting mice. These are produced using a Stacked Hourglass network trained on 15,000 hand-labeled frames.
<ul>
<li>Dimensions: (# frames) x (mouse ID) x (x, y coordinate) x (body part).
<li>Units: pixels; coordinates are relative to the entire image. Original image dimensions are 1024 x 570.
</ul>
<li><b>scores</b>: confidence estimates for the tracked keypoints.
<ul>
<li>Dimensions: (# frames) x (mouse ID) x (body part).
<li>Units: unitless, range 0 (lowest confidence) to 1 (highest confidence).
</ul>
<li> <b>annotations</b>: behaviors id as an integer annotated at each frame by a domain expert. See below for the behavior id to behavior name mappings.
<ul>
<li>Dimensions: (# frames) .
</ul>
<li><b>metadata</b>: The recorded metadata is annotator_id which is represented by an int, and the vocab, containing a dictionary which maps behavior names to integer ids in annotations.
</ul>

The 'taskprog_features' file contains the additional field:

<ul>
<li><b>features</b>: pre-computed features from a model trained with task programming on the trajectory data of the CalMS21 unlabeled videos set.
<ul>
<li>Dimensions: (# frames) x (feature dim = 32).
</li>
</ul>
</ul>

<b>NOTE:</b> for all keypoints, mouse 0 is the resident (black) mouse and mouse 1 is the intruder (white) mouse. There are 7 tracked body parts, ordered (nose, left ear, right ear, neck, left hip, right hip, tail base).

### Data overview

As described above, our dataset consists of train and test sets, which are both dictionaries of Sequences, and an accompanying vocabulary telling us which behavior is which:

In [ ]:
print("Sample dataset keys: ", list(training_data.keys())[:3])
print("Vocabulary: ", vocab)
print("Number of train Sequences: ", len(training_data))
print("Number of test Sequences: ", len(test_data))

### Sample overview

Next let's take a look at one example Sequence:

In [ ]:
sequence_names = list(training_data.keys())
sample_sequence_key = sequence_names[0]
single_sequence = training_data[sample_sequence_key]
print("Name of our sample sequence: ", sample_sequence_key)
print("Sequence keys: ", single_sequence.keys())
print("Sequence metadata: ", single_sequence['metadata'])
print(f"Number of Frames in Sequence \"{sample_sequence_key}\": ", len(single_sequence['annotations']))
print(f"Keypoints data shape of Sequence \"{sample_sequence_key}\": ", single_sequence['keypoints'].shape)

## Reshaping for Keypoint-Moseq input

Keypoint-moseq model requires 3 inputs: Coordinates, confidences, bodyparts

### Testing/Experimenting

#### Coordinates Input: Taking One sample and trying to see if we can reshape it to the shape of the input for keypoint-moseq:

Required Input: coordinates array of shape: (no. of frames x body part x no. of coordinates)

In [ ]:
training_data['task1/train/mouse001_task1_annotator1'].keys()

In [ ]:
# Shape of 'keypoints' of 1 sequence
samp_seq = training_data['task1/train/mouse001_task1_annotator1']['keypoints']
samp_seq.shape

In [ ]:
samp_seq[0,0,0,:] #First frame, resident mouse, x axis coordinate, all body parts
#First index: frame number
#Second index: mouse ID - 0:resident ; 1:intruder
#Third index: coordinate axis  -  0:x-axis ; 1:y-axis
#Fourth index: body part  -  (nose, left ear, right ear, neck, left hip, right hip, tail base)

In [ ]:
#Reshaping one of the assays to the way required by keypoint-moseq

#Splitting resident and intruder mouse keypoints:
samp_seq_resident = samp_seq[:,0,:,:]
samp_seq_intruder = samp_seq[:,1,:,:]

#Reshaping it to: frame x body part x coordinate  (no. of frames, 7, 2)
samp_seq_resident = np.transpose(samp_seq_resident, (0, 2, 1))
samp_seq_resident.shape

#### Confidences Input: For 1 assay

Required input: confidence array of shape: ( no. of frame x body part )

.Confidence score range: 0 to 1

In [ ]:
# Shape of 'scores' of 1 sequence
conf_seq = training_data['task1/train/mouse001_task1_annotator1']['scores']
conf_seq.shape

In [ ]:
# reshaping:

#splitting resident and intruder
conf_seq_res = conf_seq[:,0,:]
conf_seq_int = conf_seq[:,1,:]

conf_seq_res.shape

#### Bodyparts Input

In [ ]:
bodyparts = ['nose', 'left ear', 'right ear', 'neck', 'left hip', 'right hip', 'tail base']
bodyparts

### **Final: Getting the 3 inputs for the keypoint-moseq model**

In [ ]:
## Creating a Dictionary that maps recordings to arrays of shape (num_frames, num_keypoints, num_dimensions) for resident and intruder mouse
## This will be the 'coordinates' input for the keypoint-moseq model

coordinates_resident = {}
confidences_resident = {}
coordinates_intruder = {}
confidences_intruder = {}

for i in training_data.keys():

  coord_data = training_data[i]['keypoints']
  conf_data = training_data[i]['scores']

  reshaped_coord_data_res = np.transpose(coord_data[:,0,:,:], (0,2,1))
  reshaped_coord_data_intr = np.transpose(coord_data[:,1,:,:], (0,2,1))
  reshaped_conf_data_res = conf_data[:,0,:]
  reshaped_conf_data_intr = conf_data[:,1,:]

  coordinates_resident[i] = reshaped_coord_data_res
  coordinates_intruder[i] = reshaped_coord_data_intr
  confidences_resident[i] = reshaped_conf_data_res
  confidences_intruder[i] = reshaped_conf_data_intr

In [ ]:
bodyparts_list = ['nose', 'left ear', 'right ear', 'neck', 'left hip', 'right hip', 'tail base']
bodyparts

## Load data for the model

In [ ]:
coordinates, confidences, bodyparts = coordinates_resident, confidences_resident, bodyparts_list

# format data for the model
data, metadata = kpms.format_data(coordinates, confidences, **config())

## Fit PCA

Run the cell below to fit a PCA model to aligned and centered keypoint coordinates.

- The model is saved to ``{project_dir}/pca.p`` and can be reloaded using ``kpms.load_pca``.
- Two plots are generated: a cumulative [scree plot](https://en.wikipedia.org/wiki/Scree_plot) and a depiction of each PC, where translucent nodes/edges represent the mean pose and opaque nodes/edges represent a perturbation in the direction of the PC.
- After fitting, edit `latent_dimension` in the config. This determines the dimension of the pose trajectory used to fit keypoint-MoSeq. A good heuristic is the number of dimensions needed to explain 90% of variance, or 10 dimensions - whichever is lower.  

In [ ]:
pca = kpms.fit_pca(**data, **config())
kpms.save_pca(pca, project_dir)

kpms.print_dims_to_explain_variance(pca, 0.9)
kpms.plot_scree(pca, project_dir=project_dir)
kpms.plot_pcs(pca, project_dir=project_dir, **config())

# use the following to load an already fit model
# pca = kpms.load_pca(project_dir)

In [ ]:
# After fitting, edit latent_dimension in the config.
# This determines the dimension of the pose trajectory used to fit keypoint-MoSeq.
# A good heuristic is the number of dimensions needed to explain 90% of variance, or 10 dimensions - whichever is lower.

# I have updated latent_dim as 7 since 90% variance was explained by 7 dimensions
kpms.update_config(project_dir, latent_dim=7)

# Model fitting

Fitting a keypoint-MoSeq model involves:
1. **Initialization:** Auto-regressive (AR) parameters and syllable sequences are randomly initialized using pose trajectories from PCA.
2. **Fitting an AR-HMM:** The AR parameters, transition probabilities and syllable sequences are iteratively updated through Gibbs sampling.
3. **Fitting the full model:** All parameters, including both the AR-HMM as well as centroid, heading, noise-estimates and continuous latent states (i.e. pose trajectories) are iteratively updated through Gibbs sampling. This step is especially useful for noisy data.
4. **Extracting model results:** The learned states of the model are parsed and saved to disk for vizualization and downstream analysis.
4. **[Optional] Applying the trained model:** The learned model parameters can be used to infer a syllable sequences for additional data.

## Setting kappa

Most users will need to adjust the **kappa** hyperparameter to achieve the desired distribution of syllable durations. For this tutorial we chose kappa values that yielded a median syllable duration of 400ms (12 frames). Most users will need to tune kappa to their particular dataset. Higher values of kappa lead to longer syllables. **You will need to pick two kappas: one for AR-HMM fitting and one for the full model.**
- We recommend iteratively updating kappa and refitting the model until the target syllable time-scale is attained.  
- Model fitting can be stopped at any time by interrupting the kernel, and then restarted with a new kappa value.
- The full model will generally require a lower value of kappa to yield the same target syllable durations.
- To adjust the value of kappa in the model, use `kpms.update_hypparams` as shown below. Note that this command only changes kappa in the model dictionary, not the kappa value in the config file. The value in the config is only used during model initialization.

## Initialization

In [ ]:
# initialize the model
model = kpms.init_model(data, pca=pca, **config())

# optionally modify kappa
# model = kpms.update_hypparams(model, kappa=NUMBER)

## Fitting an AR-HMM

In addition to fitting an AR-HMM, the function below:
- generates a name for the model and a corresponding directory in `project_dir`
- saves a checkpoint every 25 iterations from which fitting can be restarted
- plots the progress of fitting every 25 iterations, including
    - the distributions of syllable frequencies and durations for the most recent iteration
    - the change in median syllable duration across fitting iterations
    - a sample of the syllable sequence across iterations in a random window

In [ ]:
num_ar_iters = 50

model, model_name = kpms.fit_model(
    model, data, metadata, project_dir,
    ar_only=True, num_iters=num_ar_iters)

## Fitting the full model

The following code fits a full keypoint-MoSeq model using the results of AR-HMM fitting for initialization. If using your own data, you may need to try a few values of kappa at this step.

In [ ]:
# load model checkpoint
model, data, metadata, current_iter = kpms.load_checkpoint(
    project_dir, model_name, iteration=num_ar_iters)

# modify kappa to maintain the desired syllable time-scale
model = kpms.update_hypparams(model, kappa=1e4)

# run fitting for an additional 500 iters
model = kpms.fit_model(
    model, data, metadata, project_dir, model_name, ar_only=False,
    start_iter=current_iter, num_iters=current_iter+500)[0]

## Sort syllables by frequency

Permute the states and parameters of a saved checkpoint so that syllables are labeled in order of frequency (i.e. so that `0` is the most frequent, `1` is the second most, and so on).

In [ ]:
# modify a saved checkpoint so syllables are ordered by frequency
kpms.reindex_syllables_in_checkpoint(project_dir, model_name);

```{warning}
Reindexing is only applied to the checkpoint file. Therefore, if you perform this step after extracting the modeling results or generating vizualizations, then those steps must be repeated.
```

## Extract model results

Parse the modeling results and save them to `{project_dir}/{model_name}/results.h5`. The results are stored as follows, and can be reloaded at a later time using `kpms.load_results`. Check the docs for an [in-depth explanation of the modeling results](https://keypoint-moseq.readthedocs.io/en/latest/FAQs.html#interpreting-model-outputs).
```
    results.h5
    ├──recording_name1
    │  ├──syllable      # syllable labels (z)
    │  ├──latent_state  # inferred low-dim pose state (x)
    │  ├──centroid      # inferred centroid (v)
    │  └──heading       # inferred heading (h)
    ⋮
```

In [ ]:
import h5py
# load the most recent model checkpoint
model, data, metadata, current_iter = kpms.load_checkpoint(project_dir, model_name)

# extract results
# results = kpms.extract_results(model, metadata, project_dir, model_name)
# results = kpms.extract_results(model, metadata, project_dir, model_name, save_results=True)
results_file = os.path.join(project_dir, model_name, "results.h5")
if os.path.exists(results_file):
    os.remove(results_file)

results = kpms.extract_results(model, metadata, project_dir, model_name, save_results=True)

### [Optional] Save results to csv

After extracting to an h5 file, the results can also be saved as csv files. A separate file will be created for each recording and saved to `{project_dir}/{model_name}/results/`.

In [ ]:
# optionally save results as csv
kpms.save_results_as_csv(results, project_dir, model_name)

# Visualization

## Trajectory plots
Generate plots showing the median trajectory of poses associated with each given syllable.

In [ ]:
kpms.generate_trajectory_plots(coordinates, results, project_dir, model_name, **config())

## Syllable Dendrogram
Plot a dendrogram representing distances between each syllable's median trajectory.

In [ ]:
kpms.plot_similarity_dendrogram(coordinates, results, project_dir, model_name, **config())